In [ ]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import pandas as pd
import os

In [ ]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/local/jdk8u222-b10'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/HDFS01/anaconda3/envs/main/bin/python'
conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.jars.packages','org.apache.spark:spark-avro_2.12:3.3.2'),
     ('spark.driver.memory', '2g'),
     ('spark.sql.repl.eagerEval.enabled','true'),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://nn01.bigdata:9083'),
     ('metastore.client.capability.check','false')
    ])
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Test_AVRO") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate();

In [ ]:
df = spark.read.csv("file:///HDFS01/airflow/notebooks/Pasit/mortgages.csv", header=True, inferSchema=True)
df = df.drop("_c0")
df.printSchema()

In [ ]:
df.write \
    .format("avro") \
    .mode("overwrite") \
    .partitionBy("qob") \
    .saveAsTable("pyspark.mortgages_avro")